### Aeon data file structure on Ceph

**Raw data**: `/ceph/aeon/aeon/data/raw/<acquisition_pc>/<experiment_name>/<acquisition_epoch>/<device>/<stream>`

e.g. `/ceph/aeon/aeon/data/raw/AEON3/social0.1/2023-12-01T14-30-34/Patch1/Patch1_90_2023-12-02T12-00-00.bin`

**Processed data (e.g. trained and exported SLEAP model)**: `/ceph/aeon/aeon/data/processed/<processed_pc>/<job_id>/<job_time>/<model_name>/frozen_graph.pb`

e.g. `/ceph/aeon/aeon/data/processed/test-node1/0000005/2023-11-30T01-29-00/topdown_multianimal_id/frozen_graph.pb`

### Reading Aeon data in Python

#### Terminology

**_Chunk Duration_**: The time duration over which experiment data files are written out. Currently, all Aeon experiments write out acquired data to files every hour (1-hour chunks).

**_Acquisition Epoch_**: One run of an experiment workflow. When an experiment workflow restarts, a new epoch starts.

E.g. `ceph/aeon/aeon/data/raw/AEON3/social0.1/2023-12-03T13-05-15` is an acquisition epoch in the Social0.1 experiment. Because the next epoch directory is `ceph/aeon/aeon/data/raw/AEON3/social0.1/2023-12-03T13-30-30`, we know this first epoch lasted only ~25 minutes.

**_Stream_**: Data that comes from a single source.

A single data file is associated with each stream, so often 'stream' and 'file' can be interchanged. If the stream comes from a harp device, the stream-file contains information about the register of the harp device which generated the stream, as well as the associated chunk datetime.

For a harp stream, the filename format is as follows:<br>
`<device><id>_<harp_register>_<datetime>` e.g. `Patch1_90_2023-12-02T12-00-00.bin`<br>
By convention, harp streams which are acquired in software start with register number '200'; e.g. the largest-blob-centroid-tracking stream filename is: `CameraTop_200*.bin`

Each stream can contain single or multi-dimensional data (e.g. a patch wheel magnetic encoder stream contains information about both the magnetic field strength and angle: however, each dimension is associated with a unique bitmask, and thus can be isolated by applying this bitmask to the stream).

**_Reader_**: A Python class whose instantiated objects each read one particular stream. Simple working principle: each `Reader` has a `read` method which takes in a single stream-file and reads the data in that file into a pandas `DataFrame` (see `aeon/io/reader.py` and `aeon/schema/*.py`).

e.g. `Encoder` readers read values from `Patch<patch_id>_<encoder_register>_<datetime>` files (these contain a patch wheel's magnetic encoder readings, to determine how much the wheel has been spun).

Whenever a new device is implemented in an Aeon experiment, a new `Reader` should be created for the acquired data, such that the data can be read and returned in the form of a pandas `DataFrame`.

**_Device_**: A collection of streams grouped together for convenience, often for related streams.

On ceph, we organize streams into device folders:<br> e.g. `ceph/aeon/aeon/data/raw/AEON3/social0.1/2023-12-01T14-30-34/Patch1` contains the patch-heartbeat stream (`Patch1_8`), the patch-beambreak stream (`Patch1_32`), the patch-pellet delivery-pin-set stream (`Patch1_35`), the patch-pellet-delivery-pin-cleared stream (`Patch1_36`), the patch-wheel-magnetic-encoder stream (`Patch1_90`), the patch-wheel-magnetic-encoder-mode stream (`Patch1_91`), the patch-feeder-dispenser-state stream (`Patch1_200`), the patch-pellet-manual-delivery stream (`Patch1_201`), the patch-missed-pellet-stream (`Patch1_202`), the patch-pellet-delivery-retry stream (`Patch1_203`), and the patch-state stream (`Patch1_State`).

In code, we create logical devices via the `Device` class (see `aeon/io/device.py`)<br>
e.g. We often define 'Patch' devices that contain `Reader` objects (in the _`register`_ attribute) that are associated with specific streams (as experimenters may not care about analyzing all streams in a `Patch` device folder on ceph), e.g. wheel-magnetic-encoder, state, pellet-delivery-pin-set, and beambreak.

One last important aspect of `Device` objects are _binder functions_: on instantiation, `Device` requires at least one argument that is a function that returns a dict of `Reader` objects (these get set into the `Device` object's `registry`). We'll explain this more in detail and show examples below.

**_Schema_**: A list of devices grouped within a `DotMap` object (see `aeon/docs/examples/schemas.py`). Each experiment is associated with a schema. If a schema changes, then the experiment neccesarily must be different (either in name or version number), as the acquired data is now different.

**_Dataset_**: All data belonging to a particular experiment. 

e.g. All data in `ceph/aeon/aeon/data/raw/AEON3/social0.1`

#### Code

With this terminology in mind, let's get to the code!

In [45]:
"""Notebook settings and imports."""

%load_ext autoreload
%autoreload 2
# %flow mode reactive

import numpy as np
import pandas as pd

from pathlib import Path

from dotmap import DotMap

import aeon
from aeon.io import reader
from aeon.io.device import Device, register
from aeon.schema import core, foraging, social
from aeon.schema.schemas import exp02

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
"""Set experiment root path and time range / set to load data"""

root = Path("/ceph/aeon/aeon/data/raw/AEON3/social0.1")
start_time = pd.Timestamp("2023-12-02 10:30:00")
end_time = pd.Timestamp("2023-12-02 12:30:00")
time_set = pd.concat(
    [
        pd.Series(pd.date_range(start_time, start_time + pd.Timedelta(hours=1), freq="1s")),
        pd.Series(pd.date_range(end_time, end_time + pd.Timedelta(hours=1), freq="1s")),
    ]
)

In [47]:
"""Creating a new `Reader` class"""

# All readers are subclassed from the base `Reader` class. They thus all contain a `read` method,
# for returning data from a file in the form of a pandas DataFrame, and the following attributes,
# which must be specified on object construction:
# `pattern`: a prefix in the filename used by `aeon.io.api.load` to find matching files to load
# `columns`: a list of column names for the returned DataFrame
# `extension`: the file extension of the files to be read

# Using these principles, we can recreate a simple reader for reading subject weight data from the
# social0.1 experiments, which are saved in .csv format.


# First, we'll create a general Csv reader, subclassed from `Reader`.
class Csv(reader.Reader):
    """Reads data from csv text files, where the first column stores the Aeon timestamp, in seconds."""

    def __init__(self, pattern, columns, extension="csv"):
        super().__init__(pattern, columns, extension)

    def read(self, file):
        return pd.read_csv(file, header=0, names=self.columns, index_col=0)


# Next, we'll create a reader for the subject weight data, subclassed from `Csv`.

# We know from our data that the files of interest start with 'Environment_SubjectWeight' and columns are:
# 1) Aeon timestamp in seconds from 1904/01/01 (1904 date system)
# 2) Weight in grams
# 3) Weight stability confidence (0-1)
# 4) Subject ID (string)
# 5) Subject ID (integer)
# Since the first column (Aeon timestamp) will be set as the index, we'll use the rest as DataFrame columns.
# And we don't need to define `read`, as it will use the `Csv` class's `read` method.


class Subject_Weight(Csv):
    """Reads subject weight data from csv text files."""

    def __init__(
        self,
        pattern="Environment_SubjectWeight*",
        columns=["weight", "confidence", "subject_id", "int_id"],
        extension="csv",
    ):
        super().__init__(pattern, columns, extension)

In [48]:
"""Loading data via a `Reader` object"""

# We can now load data by specifying a file
subject_weight_reader = Subject_Weight()
acq_epoch = "2023-12-01T14-30-34"
weight_file = root / acq_epoch / "Environment/Environment_SubjectWeight_2023-12-02T12-00-00.csv"
print("Read from a single file:")
display(subject_weight_reader.read(weight_file))

# And we can use `load` to load data across many same-stream files given a time range or time set.
print("Read from a contiguous time range:")
display(aeon.load(root, subject_weight_reader, start=start_time, end=end_time))
print("Read from a set of times:")
display(aeon.load(root, subject_weight_reader, time=time_set.values))

Read from a single file:


,weight,confidence,subject_id,int_id
3.784363e+09,29.799999,1,CAA-1120746,0
3.784363e+09,29.799999,1,CAA-1120746,0
3.784363e+09,29.799999,1,CAA-1120746,0
3.784363e+09,29.799999,1,CAA-1120746,0
3.784363e+09,29.799999,1,CAA-1120746,0
...,...,...,...,...
3.784367e+09,31.200001,1,CAA-1120747,0
3.784367e+09,31.200001,1,CAA-1120747,0
3.784367e+09,31.200001,1,CAA-1120747,0
3.784367e+09,31.200001,1,CAA-1120747,0


Read from a contiguous time range:


,weight,confidence,subject_id,int_id
time,,,,
2023-12-02 10:31:12.840000153,29.5,1,CAA-1120746,0
2023-12-02 10:31:12.940000057,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.039999962,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.099999905,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.199999809,29.5,1,CAA-1120746,0
...,...,...,...,...
2023-12-02 12:27:29.460000038,31.1,1,CAA-1120747,0
2023-12-02 12:27:29.559999943,31.1,1,CAA-1120747,0
2023-12-02 12:27:29.619999886,31.1,1,CAA-1120747,0


Read from a set of times:


,weight,confidence,subject_id,int_id
2023-12-02 10:30:00,30.000000,1.0,CAA-1120747,0.0
2023-12-02 10:30:01,30.000000,1.0,CAA-1120747,0.0
2023-12-02 10:30:02,30.000000,1.0,CAA-1120747,0.0
2023-12-02 10:30:03,30.000000,1.0,CAA-1120747,0.0
2023-12-02 10:30:04,30.000000,1.0,CAA-1120747,0.0
...,...,...,...,...
2023-12-02 13:18:25,29.799999,1.0,CAA-1120746,0.0
2023-12-02 13:18:26,29.799999,1.0,CAA-1120746,0.0
2023-12-02 13:22:05,29.900000,1.0,CAA-1120746,0.0
2023-12-02 13:22:14,29.799999,1.0,CAA-1120746,0.0


In [49]:
"""Updating a `Reader` object"""

# Occasionally, we may want to tweak the output from a `Reader` object's `read` method, or some tweaks to
# streams on the acquisition side may require us to make corresponding tweaks to a `Reader` object to
# ensure it works properly. We'll cover some of these cases here.

# 1. Column changes

# First, if we want to simply change the output from `read`, we can change the columns of an instantiated
# `Reader` object. Let's change `subject_id` to `id`, and after reading, drop the `confidence` and `int_id`
# columns.
subject_weight_reader.columns = ["weight", "confidence", "id", "int_id"]
data = subject_weight_reader.read(weight_file)
data.drop(["confidence", "int_id"], axis=1, inplace=True)
display(data)


# 2. Pattern changes

# Next, occasionally a stream's filename may change, in which case we'll need to update the `Reader`
# object's `pattern` to find the new files using `load`:


# Let's simulate a case where the old SubjectWeight stream was called Weight, and create a `Reader` class.
class Subject_Weight(Csv):
    """Reads subject weight data from csv text files."""

    def __init__(
        self,
        pattern="Environment_Weight*",
        columns=["weight", "confidence", "subject_id", "int_id"],
        extension="csv",
    ):
        super().__init__(pattern, columns, extension)


# We'll see that we can't find any files with this pattern.
subject_weight_reader = Subject_Weight()
data = aeon.load(root, subject_weight_reader, start=start_time, end=end_time)
display(data)  # empty dataframe

# But if we just update the pattern, `load` will find the files.
subject_weight_reader.pattern = "Environment_SubjectWeight*"
data = aeon.load(root, subject_weight_reader, start=start_time, end=end_time)
display(data)


# 3. Bitmask changes for Harp streams

# Lastly, some Harp streams use bitmasks to distinguish writing out different events to the same file.
# e.g. The beambreak stream `Patch<id>_32*` writes out events both for when the beam is broken and when
# it gets reset. Given a Harp stream, we can find all bitmasks associated with it, and choose which one
# to use to filter the data:

# Given a stream, we can create a `Harp` reader object to find all bitmasks associated with it.
pattern = "Patch1_32*"
event_name = "beambreak"
harp_reader = reader.Harp(pattern=pattern, columns=[event_name])
data = aeon.load(root, harp_reader, start=start_time, end=end_time)
bitmasks = np.unique(data[event_name].values)
print(f"bitmasks: {bitmasks}")

# Let's set the bitmask to '34', and create a new `Reader` object to use this.
bitmask = 34
beambreak_reader = reader.BitmaskEvent(pattern, bitmask, event_name)
bitmasked_data = aeon.load(root, beambreak_reader, start=start_time, end=end_time)

print(f"raw data:\n {data.head()}\n\n")
print(f"bitmasked data:\n {bitmasked_data.head()}")

,weight,id
3.784363e+09,29.799999,CAA-1120746
3.784363e+09,29.799999,CAA-1120746
3.784363e+09,29.799999,CAA-1120746
3.784363e+09,29.799999,CAA-1120746
3.784363e+09,29.799999,CAA-1120746
...,...,...
3.784367e+09,31.200001,CAA-1120747
3.784367e+09,31.200001,CAA-1120747
3.784367e+09,31.200001,CAA-1120747
3.784367e+09,31.200001,CAA-1120747


,weight,confidence,subject_id,int_id
time,,,,


,weight,confidence,subject_id,int_id
time,,,,
2023-12-02 10:31:12.840000153,29.5,1,CAA-1120746,0
2023-12-02 10:31:12.940000057,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.039999962,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.099999905,29.5,1,CAA-1120746,0
2023-12-02 10:31:13.199999809,29.5,1,CAA-1120746,0
...,...,...,...,...
2023-12-02 12:27:29.460000038,31.1,1,CAA-1120747,0
2023-12-02 12:27:29.559999943,31.1,1,CAA-1120747,0
2023-12-02 12:27:29.619999886,31.1,1,CAA-1120747,0


bitmasks: [32 34]
raw data:
                                beambreak
time                                    
2023-12-02 11:33:03.942463875         34
2023-12-02 11:33:03.951744080         32
2023-12-02 11:33:18.500351906         34
2023-12-02 11:33:18.503456115         32
2023-12-02 11:33:18.509632111         34


bitmasked data:
                                    event
time                                    
2023-12-02 11:33:03.942463875  beambreak
2023-12-02 11:33:18.500351906  beambreak
2023-12-02 11:33:18.509632111  beambreak
2023-12-02 11:33:18.515808104  beambreak
2023-12-02 11:33:43.750751972  beambreak


In [50]:
"""Instantiating a `Device` object"""

# A `Device` object is instantiated from a name, followed by one or more 'binder functions', which
# return a dictionary of a name paired with a `Reader` object. We call such a dictionary of `<name>:Reader`
# key-value pairs a 'registry'. Each binder function requires a  `pattern` argument, which can be used to
# set the pattern of the `Reader` object it returns. This requirement for binder functions is for allowing
# the `Device` to optionally pass its name to appropriately set the pattern of `Reader` objects it contains.


# Below are examples of "empty pattern" binder functions, where the pattern doesn't get used.
def subject_weight_binder(pattern):  # an example subject weight binder function
    return {"subject_weight": subject_weight_reader}


def subject_state_binder(pattern):  # an example subject state binder function
    return {"subject_state": reader.Subject(pattern="Environment_SubjectState*")}


d = Device("SubjectMetadata", subject_weight_binder, subject_state_binder)

# On creation, the `Device` object puts all registries into a single registry, which is  accessible via the
# `registry` attribute.
print(f"{d.registry=}")

# This is done so that we can create a 'schema' (a DotMap of a list of `Device` objects), where a `Device`
# object name is a key for the schema, and the corresponding values of the `registry` names (which are keys
# for the `Device` object) are the `Reader` objects associated with that `Device` object.

# This works because, when a list of `Device` objects are passed into the `DotMap` constructor, the
# `__iter__` method of the `Device` object returns a tuple of the object's name with its `stream`
# attribute, which is passed in directly to the DotMap constructor to create a nested DotMap:
# device_name -> stream_name -> stream `Reader` object. This is shown below:

d2 = Device("Metadata", core.metadata)  # instantiate Device from a defined binder function
schema = DotMap([d, d2])  # create schema as DotMap of list of Device objects
print(f"{schema.SubjectMetadata=}")  # Device object name as key to schema
print(f"{schema.SubjectMetadata.subject_weight=}")  # binder function name yields the Reader object
print(f"{schema.Metadata=}")  # for a singleton Device object, Device name alone yields the Reader object

d.registry={'subject_weight': <__main__.Subject_Weight object at 0x7fa4433d5f50>, 'subject_state': <aeon.io.reader.Subject object at 0x7fa43d94bcd0>}
schema.SubjectMetadata=DotMap(subject_weight=<__main__.Subject_Weight object at 0x7fa4433d5f50>, subject_state=<aeon.io.reader.Subject object at 0x7fa43d94bcd0>)
schema.SubjectMetadata.subject_weight=<__main__.Subject_Weight object at 0x7fa4433d5f50>
schema.Metadata=<aeon.io.reader.Metadata object at 0x7fa43d949350>


In [51]:
"""Nested binder functions"""

# Binder functions can return a dict whose value is actually composed of multiple, rather than a single,
# `Reader` objects. This is done by creating nested binder functions, via `register`.


# First let's define two standard binder functions, for pellet delivery trigger and beambreak events.
# In all examples below we'll define "device-name passed" binder functions, since the `Device` object which
# will be instantiated from these functions will pass its name to set the pattern of the corresponding
# Reader objects.
def pellet_trigger(pattern):
    """Pellet delivery trigger events."""
    return {"pellet_trigger": reader.BitmaskEvent(f"{pattern}_35_*", 0x80, "PelletTriggered")}


def pellet_beambreak(pattern):
    """Pellet beambreak events."""
    return {"pellet_beambreak": reader.BitmaskEvent(f"{pattern}_32_*", 0x22, "PelletDetected")}


# Next, we'll define a nested binder function for a "feeder", which returns the two binder functions above.
def feeder(pattern):
    """Feeder commands and events."""
    return register(pattern, pellet_trigger, pellet_beambreak)


# And further, we can define a higher-level nested binder function for a "patch", which includes the
# magnetic encoder values for a patch's wheel in addition to `feeder`.
def patch(pattern):
    """Data streams for a patch."""
    return register(pattern, feeder, core.encoder)


# We can now instantiate a `Device` object as done previously, from combinations of binder functions, but
# also from nested binder functions.
feeder_device = Device("Patch1", pellet_trigger, pellet_beambreak)
feeder_device_nested = Device("Patch1", feeder)
patch_device = Device("Patch1", patch)

# And we can see that `feeder_device` and `feeder_device_nested` are equivalent.
print(f"{feeder_device.registry=}")
print(f"{feeder_device_nested.registry=}")

# And `patch_device` contains the same Reader objects as these plus an `Encoder` Reader.
print(f"{patch_device.registry=}")

feeder_device.registry={'pellet_trigger': <aeon.io.reader.BitmaskEvent object at 0x7fa4430a4d90>, 'pellet_beambreak': <aeon.io.reader.BitmaskEvent object at 0x7fa4430a63d0>}
feeder_device_nested.registry={'pellet_trigger': <aeon.io.reader.BitmaskEvent object at 0x7fa4430a6910>, 'pellet_beambreak': <aeon.io.reader.BitmaskEvent object at 0x7fa4430a5d90>}
patch_device.registry={'pellet_trigger': <aeon.io.reader.BitmaskEvent object at 0x7fa4430a6050>, 'pellet_beambreak': <aeon.io.reader.BitmaskEvent object at 0x7fa443218c90>, 'Encoder': <aeon.io.reader.Encoder object at 0x7fa44321b750>}


#### Social0.1 Data Streams

Now that we've covered streams, readers, binder functions, devices, and schemas, let's build a schema for the Social0.1 Experiment!

First we'll need to know all the streams we recorded during the Social0.1 experiment: these can be found via
looking through all devices in an acqusition epoch 
(e.g. `ceph/aeon/aeon/data/raw/AEON3/social0.1/2023-12-01T14-30-34`)

And here they are: (*note: register 8 is always the harp heartbeat for any device that has this stream.*)

- Metadata.yml
- AudioAmbient (.wav)
  - .wav: raw audio
- Environment
  - BlockState
  - EnvironmentState
  - LightEvents
  - MessageLog
  - SubjectState
  - SubjectVisits
  - SubjectWeight
- CameraTop (200, 201, .avi, .csv, <model_path>)
    - 200: position
    - 201: region
- CameraNorth (avi, csv)
- CameraEast (avi, csv)
- CameraSouth (avi, csv)
- CameraWest (avi, csv)
- CameraPatch1 (avi, csv)
- CameraPatch2 (avi, csv)
- CameraPatch3 (avi, csv)
- CameraNest (avi, csv)
- ClockSynchronizer (8, 36)
    - 36: hearbeat out
- Nest (200, 201, 202, 203)
    - 200: raw weight
    - 201: tare weight
    - 202: filtered weight
    - 203: baseline weight
    - 204: subject weight
- Patch1 (8, 32, 35, 36, 87, 90, 91, 200, 201, 202, 203, State)
    - 32: beambreak
    - 35: set delivery
    - 36: clear delivery
    - 87: expansion board state
    - 90: encoder read
    - 91: encoder mode
    - 200: dispenser state
    - 201: manual delivery
    - 202: missed pellet
    - 203: retry delivery
- Patch2 (8, 32, 35, 36, 87, 90, 91, State)
- Patch3 (8, 32, 35, 36, 87, 90, 91, 200, 203, State)
- RfidEventsGate (8, 32, 35)
    - 32: entry id
    - 35: hardware notifications
- RfidEventsNest1 (8, 32, 35)
- RfidEventsNest2 (8, 32, 35)
- RfidEventsPatch1 (8, 32, 35)
- RfidEventsPatch2 (8, 32, 35)
- RfidEventsPatch3 (8, 32, 35)
- System
  - AvailableMemory
- VideoController (8, 32, 33, 34, 35, 36, 45, 52)

In [52]:
"""Creating the Social 0.1 schema"""

# Above we've listed out all the streams we recorded from during Social0.1, but we won't care to analyze all
# of them. Instead, we'll create a DotMap schema from Device objects that only contains Readers for the
# streams we want to analyze.

# We'll see both examples of binder functions we saw previously: 1. "empty pattern", and
# 2. "device-name passed".

# And we'll see both examples of instantiating Device objects we saw previously: 1. from singleton binder
# functions; 2. from multiple and/or nested binder functions.

# (Note, in the simplest case, a schema can always be created from / reduced to  "empty pattern" binder
# functions as singletons in Device objects.)

# Metadata.yml (will be a singleton binder function Device object)
# ---

core.metadata  # binder function: "device-name passed": returns a `reader.Metadata` Reader object
metadata = Device("Metadata", core.metadata)

# ---

# Environment (will be a nested, multiple binder function Device object)
# ---

# BlockState
# binder function: "device-name passed"; `pattern` will be set by `Device` object name: "Environment"
block_state_b = lambda pattern: {
    "BlockState": reader.Csv(f"{pattern}_BlockState*", ["pellet_ct", "pellet_ct_thresh", "due_time"])
}

# EnvironmentState
core.environment_state  # binder function: "device-name passed"

# Combine EnvironmentState and BlockState
env_block_state_b = lambda pattern: register(pattern, core.environment_state, block_state_b)

# LightEvents
cols = ["channel", "value"]
light_events_r = reader.Csv("Environment_LightEvents*", cols)
light_events_b = lambda pattern: {"LightEvents": light_events_r}  # binder function: "empty pattern"

# MessageLog
core.message_log  # binder function: "device-name passed"

# SubjectState
cols = ["id", "weight", "type"]
subject_state_r = reader.Csv("Environment_SubjectState*", cols)
subject_state_b = lambda pattern: {"SubjectState": subject_state_r}  # binder function: "empty pattern"

# SubjectVisits
cols = ["id", "type", "region"]
subject_visits_r = reader.Csv("Environment_SubjectVisits*", cols)
subject_visits_b = lambda pattern: {"SubjectVisits": subject_visits_r}  # binder function: "empty pattern"

# SubjectWeight
cols = ["weight", "confidence", "subject_id", "int_id"]
subject_weight_r = reader.Csv("Environment_SubjectWeight*", cols)
subject_weight_b = lambda pattern: {"SubjectWeight": subject_weight_r}  # binder function: "empty pattern"

# Nested binder fn Device object.
environment = Device("Environment", env_block_state_b, light_events_b, core.message_log)  # device name

# Separate Device object for subject-specific streams.
subject = Device("Subject", subject_state_b, subject_visits_b, subject_weight_b)

# ---

# Camera
# ---

camera_top_b = lambda pattern: {"CameraTop": reader.Video("CameraTop*")}
camera_top_pos_b = lambda pattern: {"CameraTopPos": social.Pose("CameraTop_test-node1*")}

cam_names = ["North", "South", "East", "West", "Patch1", "Patch2", "Patch3", "Nest"]
cam_names = ["Camera" + name for name in cam_names]
camera_b = [lambda pattern, name=name: {name: reader.Video(name + "*")} for name in cam_names]

camera = Device("Camera", camera_top_b, camera_top_pos_b, *camera_b)

# ---

# Nest
# ---

weight_raw_b = lambda pattern: {"WeightRaw": reader.Harp("Nest_200*", ["weight(g)", "stability"])}
weight_filtered_b = lambda pattern: {"WeightFiltered": reader.Harp("Nest_202*", ["weight(g)", "stability"])}

nest = Device(
    "Nest",
    weight_raw_b,
    weight_filtered_b,
)

# ---

# Patch
# ---

patches = ["1", "2", "3"]
patch_streams = ["32", "35", "90", "201", "202", "203", "State"]
patch_names = ["Patch" + name + "_" + stream for name in patches for stream in patch_streams]
patch_b = []
for stream in patch_names:
    if "32" in stream:
        fn = lambda pattern, stream=stream: {
            stream: reader.BitmaskEvent(stream + "*", value=34, tag="beambreak")
        }
    elif "35" in stream:
        fn = lambda pattern, stream=stream: {
            stream: reader.BitmaskEvent(stream + "*", value=1, tag="delivery")
        }
    elif "90" in stream:
        fn = lambda pattern, stream=stream: {stream: reader.Encoder(stream + "*")}
    elif "201" in stream:
        fn = lambda pattern, stream=stream: {stream: reader.Harp(stream + "*", ["manual_delivery"])}
    elif "202" in stream:
        fn = lambda pattern, stream=stream: {stream: reader.Harp(stream + "*", ["missed_pellet"])}
    elif "203" in stream:
        fn = lambda pattern, stream=stream: {stream: reader.Harp(stream + "*", ["retried_delivery"])}
    elif "State" in stream:
        fn = lambda pattern, stream=stream: {
            stream: reader.Csv(stream + "*", ["threshold", "offset", "rate"])
        }
    patch_b.append(fn)

patch = Device("Patch", *patch_b)
# ---

# Rfid
# ---

rfid_names = ["EventsGate", "EventsNest1", "EventsNest2", "EventsPatch1", "EventsPatch2", "EventsPatch3"]
rfid_names = ["Rfid" + name for name in rfid_names]
rfid_b = [lambda pattern, name=name: {name: reader.Harp(name + "*", ["rfid"])} for name in rfid_names]

rfid = Device("Rfid", *rfid_b)

# ---

In [53]:
social01 = DotMap([metadata, environment, subject, camera, nest, patch, rfid])

In [54]:
# cols = ["1", "2", "3", "4", "5"]
# r = reader.Harp("RfidEventsGate_32*", cols)
# start_time = pd.Timestamp("2023-12-02 10:30:00")
# end_time = pd.Timestamp("2023-12-02 12:30:00")
# aeon.load(root, r, start=start_time, end=end_time)

In [56]:
"""Test all readers in schema."""


def find_obj(dotmap, obj):
    """Returns a list of objects of type `obj` found in a DotMap."""
    objs = []
    for value in dotmap.values():
        if isinstance(value, obj):
            objs.append(value)
        elif isinstance(value, DotMap):
            objs.extend(find_obj(value, obj))
    return objs


readers = find_obj(social01, reader.Reader)
start_time = pd.Timestamp("2023-12-05 15:00:00")
end_time = pd.Timestamp("2023-12-07 11:00:00")
for r in readers:
    data = aeon.load(root, r, start=start_time, end=end_time)
    # assert not data.empty, f"No data found with {r}."
    print(f"\n{r.pattern}:")
    display(data.head())


Metadata:


,workflow,commit,metadata
time,,,



Environment_EnvironmentState_*:


,state
time,
2023-12-05 15:28:04.552000046,Maintenance
2023-12-05 15:30:23.199999809,Experiment



Environment_BlockState*:


...

,rfid
time,
2023-12-05 15:00:00.000000000,3784633200
2023-12-05 15:00:00.001503944,3784633200
2023-12-05 15:00:01.000000000,3784633201
2023-12-05 15:00:01.001503944,3784633201
2023-12-05 15:00:02.000000000,3784633202



RfidEventsPatch2*:


,rfid
time,
2023-12-05 15:02:33.719103813,977200010163729
2023-12-05 15:02:34.209599972,977200010163729
2023-12-05 15:02:34.608064175,977200010163729
2023-12-05 15:02:35.006527901,977200010163729
2023-12-05 15:02:35.251743793,977200010163729



RfidEventsPatch3*:


,rfid
time,
2023-12-05 15:02:19.841599941,977200010164323
2023-12-05 15:02:20.271039963,977200010164323
2023-12-05 15:02:20.731232166,977200010164323
2023-12-05 15:02:21.130015849,977200010164323
2023-12-05 15:02:21.896927834,977200010164323
